In [ ]:
install.packages('DescTools')

In [ ]:
library(stringr)
library(data.table)
#library(DescTools)
library(ggplot2)
library(dplyr)
covars <- fread("AoU_98K_covariates.tsv")
icd <- fread("icd_cohort_processed.txt")
colnames(icd)[2:1854] <- paste0("Phe_", str_replace(colnames(icd)[2:1854], "\\.", "_"))
head(colnames(icd))

In [ ]:
#args <- phecode <- "Baso_Fra_Mean_INT"
#source('combine_eval_PRScs.R')

In [ ]:
source("eval_functions.R")
#combine_plink_scores("LDLC_Mean_INT", "^PRScsx.*.sscore$", "PRScsx", "computed_scores/")

In [ ]:
combine_plink_scores("Hemoglobin_Mean_INT", "LDp", "LDpred", "computed_scores/")

In [ ]:
combine_plink_scores("Baso_Fra_Mean_INT", "^PRScsx_", "PRScsx1", "computed_scores/")

In [ ]:
combine_plink_scores("mean_Height_INT", "^PRS_", "PRScs", "computed_scores/")

In [ ]:
combine_plink_scores("Hemoglobin_Mean_INT", "^PRScsx_", "PRScsx", "computed_scores/")

In [ ]:
#combine_plink_scores("Baso_Fra_Mean_INT", "^PRScsx.*.sscore$", "PRScsx", "computed_scores/")
combine_plink_scores("CRP_dL_Mean_INT", "^PRScsx.*.sscore$", "PRScsx", "computed_scores/")
combine_plink_scores("proBNP_Mean_INT", "^PRScsx.*.sscore$", "PRScsx", "computed_scores/")

In [ ]:
combine_plink_scores("Phe_428_1", "^PRScsx.*.sscore$", "PRScsx", "computed_scores/")
combine_plink_scores("Phe_427_21", "^PRScsx.*.sscore$", "PRScsx", "computed_scores/")
combine_plink_scores("Phe_250_1", "^PRScsx.*.sscore$", "PRScsx", "computed_scores/")
combine_plink_scores("Phe_250_2", "^PRScsx.*.sscore$", "PRScsx", "computed_scores/")
combine_plink_scores("Phe_411_2", "^PRScsx.*.sscore$", "PRScsx", "computed_scores/")
combine_plink_scores("Phe_411_3", "^PRScsx.*.sscore$", "PRScsx", "computed_scores/")


In [ ]:
args <- phecode <- "Baso_Fra_Mean_INT"
source('combine_ldpred.R')


In [ ]:
#args <- dir <- "ldpred_compares"
#source('combine_ldpred_comp.R')

In [ ]:
dat <- icd %>% left_join(covars) %>% left_join(sire)
#colnames(dat)
table(dat$race, dat$ethnicity)

In [ ]:
sire <- fread("self_report_demographics.txt") 

In [ ]:
table(sire$sire, sire$ancestry_pred_other)
table(sire$hispanic, sire$ancestry_pred_other)

sire %>% filter(ancestry_pred_other == "oth") %>%   
    group_by(sire) %>%          # now required with changes to dplyr::count()
    summarize(n = n())

round(prop.table(table(sire$race, sire$ancestry_pred_other),1),2)
round(prop.table(table(sire$race, sire$ancestry_pred_other),2),2)

In [ ]:
dat <- icd %>% select("person_id", all_of(phecode)) %>% left_join(covars) %>%   left_join(score0, by = c(person_id = "IID"))

In [ ]:
table(dat[[phecode]], dat$ancestry_pred_other)

In [ ]:
compare_cstat_prscs <- function(phecode, pop){
file1 <- paste0(phecode, "cstat_res.txt")
covs <- "age + is_male + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10"
dat_anc <- dat[dat$ancestry_pred_other == pop, ]
baseline <- glm(as.formula(paste(phecode, covs, sep=" ~ ")), data = dat_anc)
META_afr <- glm(as.formula(paste(phecode, paste(c("SCORE1_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(META_afr))

META_amr <- glm(as.formula(paste(phecode, paste(c("SCORE2_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(META_amr))

META_eur <- glm(as.formula(paste(phecode, paste(c("SCORE3_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(META_eur))

AFR_afr <- glm(as.formula(paste(phecode, paste(c("SCORE4_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(AFR_afr))

HIS_amr <- glm(as.formula(paste(phecode, paste(c("SCORE5_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(HIS_amr))

EUR_eur <- glm(as.formula(paste(phecode, paste(c("SCORE6_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(EUR_eur))
write(paste(phecode, pop, Cstat(baseline), Cstat(META_afr), Cstat(META_amr), Cstat(META_eur), Cstat(AFR_afr), Cstat(HIS_amr), Cstat(EUR_eur)), file = file1, append = T)
}


In [ ]:
ggplot(covars) + geom_histogram(aes(x = age, color = ancestry_pred_other))

In [ ]:
#T1DLDp <- fread("Phe_250_1cstat_LDp_res.txt")
#head(T1DLDp)

In [ ]:
#T1D_prscs <- fread("Phe_250_1cstat_res.txt")
#head(T1D_prscs)

In [ ]:
afibLDp <- fread("Phe_427_21cstat_LDp_res.txt")
head(afibLDp)

In [ ]:
afib_prscs <- fread("Phe_427_21cstat_res.txt")
head(afib_prscs)

In [ ]:
pop <- "afr"
covs <- "age + is_male + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10"
dat_anc <- dat[dat$ancestry_pred_other == pop, ]
baseline <- glm(as.formula(paste(phecode, covs, sep=" ~ ")), data = dat_anc)

summary(baseline)
mle_inf <- glm(as.formula(paste(phecode, paste(c("SCORE1_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(mle_inf))

mle_auto <- glm(as.formula(paste(phecode, paste(c("SCORE2_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(mle_auto))

nomle_inf <- glm(as.formula(paste(phecode, paste(c("SCORE3_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(nomle_inf))

nomle_auto <- glm(as.formula(paste(phecode, paste(c("SCORE4_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(nomle_auto))

In [ ]:
covs <- "is_male + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10"
baseline <- glm(as.formula(paste(phecode, covs, sep=" ~ ")), data = dat_anc)

mle_inf <- glm(as.formula(paste(phecode, paste(c("SCORE1_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(mle_inf))

mle_auto <- glm(as.formula(paste(phecode, paste(c("SCORE2_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(mle_auto))

nomle_inf <- glm(as.formula(paste(phecode, paste(c("SCORE3_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(nomle_inf))

nomle_auto <- glm(as.formula(paste(phecode, paste(c("SCORE4_SUM", covs), collapse=" + "), sep=" ~ ")), data = dat_anc)
print(summary(nomle_auto))

In [ ]:
library(data.table)
library(ggplot2)
library(dplyr)
library(tidyr)

In [ ]:
a_long <- a %>% pivot_longer(cols = c("baseline","META_afr", "META_amr", "META_eur", "AFR_afr", "HIS_amr", "EUR_eur"), names_to = "score", values_to = "AUC")

In [ ]:
ggplot(a_long %>% filter(anc != "eas")) + geom_boxplot(aes(x = phecode, y = AUC, color = score)) + facet_wrap(~anc) + scale_x_discrete(guide = guide_axis(n.dodge = 2)) 

In [ ]:
ggplot(dat %>% filter(!ancestry_pred_other %in% c("mid", "sas", "eas"))) + geom_density(aes(x = SCORE5_SUM, color = ancestry_pred_other))